In [1]:
import pandas as pd
import numpy as np
import ast

import warnings
warnings.filterwarnings('ignore')

import nltk 
import re
import sys
import os
from itertools import dropwhile
#import postagger
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import spacy
from spacy import displacy

In [2]:
nlp = spacy.load('en')

In [3]:
def parse_dependency_new(x):
    
    def existence_checking(p):
        try:
            temp[p]
            y = True
        except:
            y = False
        return y

    def check(p1,p2):
        try:
            graph[p2][p1]
            y = True
        except:
            y = False
        return y

    doc = nlp(x)
    temp = {}
    for token in doc:
        if not existence_checking(token.text):
            temp[token.idx] = token.head.idx
        else:
            temp[token.idx] = token.head.idx
    
    root_dic = {}        
    for token in doc:
        if token.dep_ == 'ROOT':
            root_dic[token.idx] = {}
            
    reverse_dic = {}
    keys = list(set([k for k,v in temp.items()]))
    values = list(set([v for k,v in temp.items()]))
    
    graph = {}
    for k in values:
        temp_ = {}
        for v in keys:
            if v != k:
                if temp[v] == k:
                    temp_[v] = {}
        graph[k] = temp_
    
    for k,v in graph.items():
        for k1, v1 in graph.items():
            if check(k,k1):
                graph[k1][k] = graph[k] 
    
    graph_ = {}
    for k in root_dic.keys():
        graph_[k] = graph[k]
    
    return graph_

In [4]:
def findKeys(data, key,result):    
    if key in list(data.keys()):
        #print(key,"====",data)
        result.append(data[key])
    else:
        for i in list(data.keys()):
            if type(data[i]) == dict:
                #print(i,data[i])
                findKeys(data[i],key,result)
    return result

In [5]:
def findKeys_K(data,key_,word_id_dic):
    key_id_lis = []
    for k,v in word_id_dic.items():
        if v == key_:
            key_id_lis.append(k)
    result_all = []
    for key in key_id_lis:
        result = []
        result_all.append(findKeys(data,key,result))
    return result_all

In [27]:
def negation_existence(x, key_w):
    x = x.lower()
    x = x.replace('~','')
    x = x.replace('$','')
    doc = nlp(x)
    
    word_id_dic = {}
    for token in doc:
        word_id_dic[token.idx] = token.text
    
    output_lis = []
    
    for token in doc:
        if token.text in ['no','not',"n't","wouldn't",'never','nobody','nothing','neither','nowhere','noone',
                         'no-one','hardly','scarcely','barely']:
            try:
                if token.dep_ == 'det':
                    if token.head.text == key_w:
                        y = 1
                    else:
                        y = 0
                else:
                    try:
                        if token.dep_ == 'neg' or token.head.pos_ == 'VERB':
                            
                            g = parse_dependency_new(x)
                            g1 = findKeys_K(g,token.head.text,word_id_dic)
                            
                            if len(g1) != 0:
                                len_lis = []
                                for j in range(len(g1)):
                                    g2 = findKeys_K(g1[j][0],key_w, word_id_dic)
                                    len_lis.append(len(g2[0]))
                                
                                if any(len_lis) !=0:
                                    y = 1
                                else:
                                    y = 0
                            else:
                                y = 0
                        else:
                            y = 0
                    except:
                        pass
                output_lis.append(y)
            except:
                pass
        else:
            pass
    if any(output_lis) == 1:
        out = 1
    else:
        out = 0
    return out

In [29]:
negation_existence("I like tea, but I don't drink coffee",'tea')

0

In [30]:
negation_existence("I like tea, but I don't drink coffee",'coffee')

1

In [31]:
doc = nlp("I like tea, but I don't drink coffee")

In [32]:
displacy.render(doc,style='dep',jupyter=True)